In [1]:
import numpy as np
from tqdm import tqdm
import os

io_path = '/dk/lcj_rag/pyserini-autofusion/experiments/runs'

depth=1000
#  alpha*bf + (1-alpha)*bm25
alpha = 0.5
nq = -1
k=100

bf_trec_path = f'{io_path}/base_trec/run.dpr.curated-train.bm25.trec'
bm25_trec_path = f'{io_path}/base_trec/run.dpr.curated-train.multi.bf.trec'
cache_path = '/dk/lcj_rag/pyserini-autofusion/experiments/cache/'
fusion_tag = 'fusion'


In [2]:

# for alpha in tqdm([0, 0.5, 1.00]):
for alpha in tqdm([0, 1.00]):
#     alpha = round(alpha, 2)
    alpha = f'{alpha:.2f}'
    print(alpha)
    train_out_trec_path = f'{io_path}/curated_train/run.dpr.curated-train.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.trec'
    train_out_json_path = f'{io_path}/curated_train/run.dpr.curated-train.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.json'
    cmd0 = f'python -m pyserini.autofusion \
            --runs {bf_trec_path} {bm25_trec_path} \
            --runtag {fusion_tag} \
            --method interpolation  \
            --alpha {alpha} \
            --depth {depth}  \
            --nq {nq}  \
            --k {k} \
            --output {train_out_trec_path}'
    # print(cmd0)
    os.system(cmd0)


  0%|          | 0/2 [00:00<?, ?it/s]

0
num_queries: 1121000


 50%|█████     | 1/2 [00:26<00:26, 26.74s/it]

1.0
num_queries: 1121000


100%|██████████| 2/2 [00:55<00:00, 27.80s/it]


In [4]:
nq = -1
# for alpha in tqdm([0]):
for alpha in tqdm([0, 1.00]):
    # alpha = round(alpha, 2)
    alpha = f'{alpha:.1f}'
    # alphafile = f'/dk/lcj_rag/pyserini-autofusion/experiments/runs/curated_train/alpha-{alpha}.csv'
    train_out_trec_path = f'{io_path}/curated_train/run.dpr.curated-train.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.trec'
    train_out_json_path = f'{io_path}/curated_train/run.dpr.curated-train.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.json'
    # cmd0 = f'python -m pyserini.autofusion \
    #         --runs {bf_trec_path} {bm25_trec_path} \
    #         --runtag {fusion_tag}-custom \
    #         --method custom  \
    #         --alphafile {alphafile} \
    #         --depth {depth}  \
    #         --nq {nq}  \
    #         --k {k} \
    #         --output {train_out_trec_path}'
    # # print(cmd0)
    # os.system(cmd0)

    # cmd1 = f'python -m pyserini.eval.convert_trec_run_to_dpr_retrieval_run \
    #     --index wikipedia-dpr-100w \
    #     --topics dpr-curated-train \
    #     --input {train_out_trec_path} \
    #     --output {train_out_json_path}'
    # os.system(cmd1)

    cache_path = '/dk/lcj_rag/pyserini-autofusion/experiments/cache/'
    cmd1 = f'python -m pyserini.eval.convert_trec_run_to_dpr_retrieval_run_on_cache \
    --index wikipedia-dpr-100w \
    --topics dpr-curated-train \
    --gt "" {cache_path}curated_train_qid_gt_set.pkl \
    --input {train_out_trec_path} \
    --output {train_out_json_path}'
    os.system(cmd1)
    

  0%|          | 0/2 [00:00<?, ?it/s]

resultPath: /root/.cache/pyserini/topics-and-qrels/topics.dpr.curated.train.txt
checking path: /dk/rag-dataset/indexes/lucene-index.wikipedia-dpr-100w.20210120.d1b9e6...


Jul 01, 2024 11:58:57 AM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false
 50%|█████     | 1/2 [00:30<00:30, 30.28s/it]

resultPath: /root/.cache/pyserini/topics-and-qrels/topics.dpr.curated.train.txt
checking path: /dk/rag-dataset/indexes/lucene-index.wikipedia-dpr-100w.20210120.d1b9e6...


Jul 01, 2024 11:59:27 AM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false
100%|██████████| 2/2 [01:00<00:00, 30.48s/it]


In [5]:
filename = f'{io_path}/curated_train/run.dpr.curated-train.{fusion_tag}.all-alpha.nq-{nq}.k-{k}.res.test'
train_out_json_path = f'{io_path}/curated_train/run.dpr.curated-train.{fusion_tag}.alpha-?.nq-{nq}.k-{k}.json'

cmd2 = f'python -m pyserini.eval.evaluate_dpr_retrieval_all_alpha \
    --retrieval {train_out_json_path} \
    --topk 100  \
    --traindata False'
res = os.system(cmd2 + f'1> {filename}')
print(filename)

  0%|          | 0/3 [00:00<?, ?it/s]
1121it [00:00, 18733.67it/s]
 33%|███▎      | 1/3 [00:00<00:01,  2.00it/s]
1121it [00:00, 15470.41it/s]
 67%|██████▋   | 2/3 [00:00<00:00,  2.13it/s]
1121it [00:00, 18728.82it/s]
100%|██████████| 3/3 [00:01<00:00,  2.25it/s]


/dk/lcj_rag/pyserini-autofusion/experiments/runs/curated_train/run.dpr.curated-train.fusion.all-alpha.nq--1.k-100.res.test
